In [1]:
# Necesarry Imports
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

Preprocess Data and Create Labels and Features

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/13O-5I0-hh__8n53OQF4TCDkEMeEm33y98EutsDE1yfE/export?format=csv")

actions =  np.array(list(df["actions_label"]))
label_map = {label:num for num, label in enumerate(actions)}

DATA_PATH = os.path.join('TakeStoreData') 
# Videos are going to be 30 frames in length
sequence_length = 30

In [5]:
sequences, labels = [], []
index = 0
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH,  f"{index}. {action}"))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, f"{index}. {action}", str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    index+=1

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [12]:
X = np.array(sequences)

In [13]:
y = to_categorical(labels).astype(int)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Build and Train LSTM Neural Network

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import TensorBoard

In [8]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [9]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
# accuracy with train data
res = model.predict(X_train)
count=0
for i in range(0,X_train.shape[0]): 
    if actions[np.argmax(res[i])] == actions[np.argmax(y_train[i])]:
        count+=1
print(count*100/X_train.shape[0])

Save Weights

In [ ]:
model.save('model.h5')

Evaluation using Confusion Matrix and Accuracy

In [16]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat_Test = model.predict(X_test)
yhat_Train = model.predict(X_train)

In [29]:
ytrue_Test = np.argmax(y_test, axis=1).tolist()
yhat_Test = np.argmax(yhat_Test, axis=1).tolist()

In [32]:
ytrue_Train = np.argmax(y_train, axis=1).tolist()
yhat_Train = np.argmax(yhat_Train, axis=1).tolist()

In [ ]:
print("Test Data Accuracy= "+str(accuracy_score(ytrue_Test, yhat_Test)))
print("Train Data Accuracy= "+str(accuracy_score(ytrue_Train, yhat_Train)))

In [ ]:
print(multilabel_confusion_matrix(ytrue_Test, yhat_Test))

In [ ]:
print(multilabel_confusion_matrix(ytrue_Train, yhat_Train))